<a href="https://colab.research.google.com/github/okdiplodok/Multimodal-emotion-recognition/blob/master/Bert_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

base code borrowed from [this Google Colab Notebook](https://colab.research.google.com/github/google-research/bert/blob/master/predicting_movie_reviews_with_bert_on_tf_hub.ipynb).

Refactored by [Shuyi Wang](https://www.linkedin.com/in/shuyi-wang-b3955026/) and [Yan Sun](https://github.com/SunYanCN)

Please refer to [this Medium Article](https://medium.com/@wshuyi/how-to-do-text-binary-classification-with-bert-f1348a25d905) for detailed information.



In [1]:
!pip install bert-text

     |████████████████████████████████| 345.2MB 45kB/s 
     |████████████████████████████████| 71kB 28.9MB/s 


In [2]:
!wget https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
!wget https://raw.githubusercontent.com/google-research/bert/master/modeling.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/optimization.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/run_classifier.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/tokenization.py 

--2019-06-05 12:15:42--  https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.25.112, 2404:6800:4004:80d::2010
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.25.112|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 662903077 (632M) [application/zip]
Saving to: ‘multi_cased_L-12_H-768_A-12.zip’

multi_cased_L-12_H- 100%[===================>] 632.19M  67.6MB/s    in 8.6s    

2019-06-05 12:15:51 (73.4 MB/s) - ‘multi_cased_L-12_H-768_A-12.zip’ saved [662903077/662903077]

--2019-06-05 12:15:52--  https://raw.githubusercontent.com/google-research/bert/master/modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 379

In [0]:
import modeling
import optimization
import run_classifier
import tokenization

In [0]:
import zipfile
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score, roc_auc_score
import os
import datetime

In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [0]:
folder = 'model_folder'
with zipfile.ZipFile("multi_cased_L-12_H-768_A-12.zip","r") as zip_ref:
    zip_ref.extractall(folder)

In [6]:
BERT_MODEL = 'multi_cased_L-12_H-768_A-12'
BERT_PRETRAINED_DIR = f'{folder}/multi_cased_L-12_H-768_A-12'
OUTPUT_DIR = f'{folder}/outputs'
print(f'>> Model output directory: {OUTPUT_DIR}')
print(f'>>  BERT pretrained directory: {BERT_PRETRAINED_DIR}')

>> Model output directory: model_folder/outputs
>>  BERT pretrained directory: model_folder/multi_cased_L-12_H-768_A-12


In [0]:
dataset = pd.read_csv('bert/refined_for_bert.csv', header=0, sep=';')[['subs', 'text']]
dataset['Label'] = LabelEncoder().fit_transform(dataset['subs'])
dataset.head()


,subs,text,Label
0,anger,Kan ik hier nog betalen of hoe zit dat?,0
1,neutrality,Het is eigenlijk een heel algemeen en beschouw...,4
2,surprise,Hee. Is Sjoerd al weg?,6
3,happiness,Ik ben blij dat ze gelukkig is met Anton en ju...,3
4,neutrality,Ik weet het niet.,4


In [0]:
df = dataset[['text', 'Label']]

In [0]:
df['Label'].unique()

array([0, 4, 6, 3, 5, 2, 1])

In [0]:
X_train, X_test, y_train, y_test = train_test_split(df['text'].values, df['Label'].values, test_size=0.2, random_state=42)

In [12]:
def create_examples(lines, set_type, labels=None):
#Generate data for the BERT model
    guid = f'{set_type}'
    examples = []
    if guid == 'train':
        for line, label in zip(lines, labels):
            text_a = line
            label = str(label)
            examples.append(
              run_classifier.InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
    else:
        for line in lines:
            text_a = line
            label = '0'
            examples.append(
              run_classifier.InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
    return examples

# Model Hyper Parameters
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
LEARNING_RATE = 1e-5
NUM_TRAIN_EPOCHS = 10.0
WARMUP_PROPORTION = 0.1
MAX_SEQ_LENGTH = 50
# Model configs
SAVE_CHECKPOINTS_STEPS = 100000 #if you wish to finetune a model on a larger dataset, use larger interval
# each checpoint weights about 1,5gb
ITERATIONS_PER_LOOP = 100000
NUM_TPU_CORES = 8
VOCAB_FILE = os.path.join(BERT_PRETRAINED_DIR, 'vocab.txt')
CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')
DO_LOWER_CASE = BERT_MODEL.startswith('uncased')

label_list = [str(num) for num in range(7)]
#label_list = [str(num) for num in range(3)] <- uncomment for the sentiment analysis experiment and comment the previous line
tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=DO_LOWER_CASE)
train_examples = create_examples(X_train, 'train', labels=y_train)

tpu_cluster_resolver = None #Since training will happen on GPU, we won't need a cluster resolver
#TPUEstimator also supports training on CPU and GPU. You don't need to define a separate tf.estimator.Estimator.
run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=OUTPUT_DIR,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

num_train_steps = int(
    len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

model_fn = run_classifier.model_fn_builder(
    bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
    num_labels=len(label_list),
    init_checkpoint=INIT_CHECKPOINT,
    learning_rate=LEARNING_RATE,
    num_train_steps=num_train_steps,
    num_warmup_steps=num_warmup_steps,
    use_tpu=False, #If False training will fall on CPU or GPU, depending on what is available  
    use_one_hot_embeddings=True)

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=False, #If False training will fall on CPU or GPU, depending on what is available 
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Using config: {'_model_dir': 'model_folder/outputs', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 100000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f1f0ecdfc88>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is

In [13]:
print('Please wait...')
train_features = run_classifier.convert_examples_to_features(
    train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
print('>> Started training at {} '.format(datetime.datetime.now()))
print('  Num examples = {}'.format(len(train_examples)))
print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
tf.logging.info("  Num steps = %d", num_train_steps)
train_input_fn = run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=True)
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print('>> Finished training at {}'.format(datetime.datetime.now()))

Please wait...
INFO:tensorflow:Writing example 0 of 1239
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: train
INFO:tensorflow:tokens: [CLS] Met hoe ##ve ##el pass ##ie je over ##al voor gaat . Ook voor onze liefde . [SEP]
INFO:tensorflow:input_ids: 101 19728 22735 10612 10570 23392 10400 10144 10491 10415 10423 25397 119 16434 10423 41208 98196 119 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:label: 2 (id = 2)
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: train
INFO:tensorflow:tokens: [CLS] De zit achter me aan . Miss ##chien word ik gear ##reste ##erd . [SEP]
INFO:tensorflow:input_ids: 101 10190 84265 32202 10911 10727 119 14247 95584 12307 47458 62559 69829 13819 119 102 0 0 0 0 0

In [0]:
def input_fn_builder(features, seq_length, is_training, drop_remainder):
  """Creates an `input_fn` closure to be passed to TPUEstimator."""

  all_input_ids = []
  all_input_mask = []
  all_segment_ids = []
  all_label_ids = []

  for feature in features:
    all_input_ids.append(feature.input_ids)
    all_input_mask.append(feature.input_mask)
    all_segment_ids.append(feature.segment_ids)
    all_label_ids.append(feature.label_id)

  def input_fn(params):
    """The actual input function."""
    print(params)
    batch_size = 128

    num_examples = len(features)

    d = tf.data.Dataset.from_tensor_slices({
        "input_ids":
            tf.constant(
                all_input_ids, shape=[num_examples, seq_length],
                dtype=tf.int32),
        "input_mask":
            tf.constant(
                all_input_mask,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "segment_ids":
            tf.constant(
                all_segment_ids,
                shape=[num_examples, seq_length],
                dtype=tf.int32),
        "label_ids":
            tf.constant(all_label_ids, shape=[num_examples], dtype=tf.int32),
    })

    if is_training:
      d = d.repeat()
      d = d.shuffle(buffer_size=100)

    d = d.batch(batch_size=batch_size, drop_remainder=drop_remainder)
    return d

  return input_fn

In [15]:
predict_examples = create_examples(X_test, 'test')

predict_features = run_classifier.convert_examples_to_features(
    predict_examples, label_list, MAX_SEQ_LENGTH, tokenizer)

predict_input_fn = input_fn_builder(
    features=predict_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

result = estimator.predict(input_fn=predict_input_fn)

INFO:tensorflow:Writing example 0 of 310
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: test
INFO:tensorflow:tokens: [CLS] Sc ##hat , ik v ##rees dat het iets ge ##com ##plice ##erde ##r ligt . [SEP]
INFO:tensorflow:input_ids: 101 55260 19180 117 47458 190 76589 10527 10187 41610 46503 22530 62631 15676 10129 18616 119 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:label: 0 (id = 0)
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: test
INFO:tensorflow:tokens: [CLS] Ik hou van ' m , ik hou van jo ##u . Maa ##k me dat een sl ##echt mens ? Ne ##e ! [SEP]
INFO:tensorflow:input_ids: 101 74122 109601 10145 112 181 117 47458 109601 10145 12541 10138 119 95057 10174 10911 10527 10200 38523 333

In [16]:
preds = []
for prediction in result:
      preds.append(np.argmax(prediction['probabilities']))

{}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 50)
INFO:tensorflow:  name = input_mask, shape = (?, 50)
INFO:tensorflow:  name = label_ids, shape = (?,)
INFO:tensorflow:  name = segment_ids, shape = (?, 50)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (119547, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/

In [0]:
test_set['predictions'] = preds

In [0]:
test_set.to_csv('BERT_predictions.csv', header=True, index=False)

In [0]:
test_set = pd.read_csv('annotation_results.csv', header=0)[['subs', 'text']]

In [0]:
test_set['Label'] = test_set['subs'].map({'anger': 0,
                                         'disgust': 1,
                                         'fear': 2,
                                         'happiness': 3,
                                         'neutrality': 4, 
                                         'sadness': 5,
                                         'surprise': 6})

In [22]:
X_test = test_set['text']
predict_examples = create_examples(X_test, 'test')

predict_features = run_classifier.convert_examples_to_features(
    predict_examples, label_list, MAX_SEQ_LENGTH, tokenizer)

predict_input_fn = input_fn_builder(
    features=predict_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

result = estimator.predict(input_fn=predict_input_fn)

INFO:tensorflow:Writing example 0 of 636
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: test
INFO:tensorflow:tokens: [CLS] Ik heb iets anders ontdekt . [SEP]
INFO:tensorflow:input_ids: 101 74122 79807 41610 41087 59499 119 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:label: 0 (id = 0)
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: test
INFO:tensorflow:tokens: [CLS] Ji ##j , fie ##ts als ##je ##bli ##ef ##t even naar het centrum voor de ball ##onne ##n . [SEP]
INFO:tensorflow:input_ids: 101 30822 10418 117 29959 10806 10223 10381 48850 16822 10123 13246 11422 10187 21203 10423 10104 20724 29129 10115 119 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [23]:
preds = []
pred_vectors = []
for prediction in result:
  pred_vectors.append(prediction['probabilities'])
  preds.append(np.argmax(prediction['probabilities']))

{}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Running infer on CPU
INFO:tensorflow:*** Features ***
INFO:tensorflow:  name = input_ids, shape = (?, 50)
INFO:tensorflow:  name = input_mask, shape = (?, 50)
INFO:tensorflow:  name = label_ids, shape = (?,)
INFO:tensorflow:  name = segment_ids, shape = (?, 50)
INFO:tensorflow:**** Trainable Variables ****
INFO:tensorflow:  name = bert/embeddings/word_embeddings:0, shape = (119547, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/token_type_embeddings:0, shape = (2, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/position_embeddings:0, shape = (512, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/beta:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/embeddings/LayerNorm/gamma:0, shape = (768,), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/layer_0/attention/self/query/kernel:0, shape = (768, 768), *INIT_FROM_CKPT*
INFO:tensorflow:  name = bert/encoder/

In [0]:
y_test = test_set['Label']
print(classification_report(y_test, preds))

              precision    recall  f1-score   support

           0       0.14      0.32      0.20        76
           1       0.14      0.02      0.04        41
           2       0.21      0.07      0.10        61
           3       0.34      0.44      0.38        71
           4       0.54      0.41      0.47       263
           5       0.28      0.36      0.32        66
           6       0.30      0.33      0.31        58

    accuracy                           0.33       636
   macro avg       0.28      0.28      0.26       636
weighted avg       0.36      0.33      0.33       636



In [0]:
# run this part for the sentiment analysis experiment
sentiment_indices = {'anger': 'neg',
                     'disgust': 'neg',
                     'fear': 'neg',
                     'happiness': 'pos',
                     'neutrality': 'neu',
                     'sadness': 'neg',
                     'surprise': 'pos'}

In [8]:
# run this part for the sentiment analysis experiment
dataset = pd.read_csv('refined_for_bert.csv', header=0, sep=';')[['subs', 'text']]
dataset['subs'] = dataset['subs'].map(sentiment_indices)
dataset['Label'] = LabelEncoder().fit_transform(dataset['subs'])
dataset.head()

,subs,text,Label
0,neg,Kan ik hier nog betalen of hoe zit dat?,0
1,neu,Het is eigenlijk een heel algemeen en beschouw...,1
2,pos,Hee. Is Sjoerd al weg?,2
3,pos,Ik ben blij dat ze gelukkig is met Anton en ju...,2
4,neu,Ik weet het niet.,1


In [10]:
df['Label'].unique()

array([0, 1, 2])

In [17]:
# for the sentiment analysis experiment
print(classification_report(y_test,preds))

              precision    recall  f1-score   support

           0       0.56      0.66      0.60       148
           1       0.35      0.22      0.27        69
           2       0.40      0.41      0.41        93

    accuracy                           0.48       310
   macro avg       0.44      0.43      0.43       310
weighted avg       0.47      0.48      0.47       310



In [0]:
# for the sentiment analysis experiment
test_set['Label'] = test_set['subs'].map(sentiment_indices)
test_set['Label'] = test_set['Label'].map({'pos': 2,
                                          'neu': 1,
                                          'neg': 0})

In [28]:
# for the sentiment analysis experiment
y_test = test_set['Label']
print(classification_report(y_test,preds))

              precision    recall  f1-score   support

           0       0.48      0.68      0.56       244
           1       0.56      0.27      0.37       263
           2       0.29      0.36      0.32       129

    accuracy                           0.45       636
   macro avg       0.44      0.44      0.42       636
weighted avg       0.47      0.45      0.43       636



In [25]:
test_set

,subs,text,Label
0,neutrality,Ik heb iets anders ontdekt.,neu
1,neutrality,"Jij, fiets alsjeblieft even naar het centrum v...",neu
2,sadness,Ook voor ons. Wij kregen een kindje en... toen...,neg
3,neutrality,Dank je wel.,neu
4,neutrality,Het is ook belangrijk dat iemand 't hoofd koel...,neu
5,anger,Als dat echt zo was dan had je me dit nooit ge...,neg
6,neutrality,"Hee, ik heb een nieuwe locatie voor de bruilof...",neu
7,sadness,"Tot de dood ons scheidt, dat duurt niet lang, he.",neg
8,neutrality,Ze doet dit voor Bing. Ze zegt 't niet uit zic...,neu
9,anger,Maar hoe... Dit kan niet. Kennelijk wel.,neg


In [0]:
test_set

,subs,text,Label
0,neutrality,Ik heb iets anders ontdekt.,4
1,neutrality,"Jij, fiets alsjeblieft even naar het centrum v...",4
2,sadness,Ook voor ons. Wij kregen een kindje en... toen...,5
3,neutrality,Dank je wel.,4
4,neutrality,Het is ook belangrijk dat iemand 't hoofd koel...,4
5,anger,Als dat echt zo was dan had je me dit nooit ge...,0
6,neutrality,"Hee, ik heb een nieuwe locatie voor de bruilof...",4
7,sadness,"Tot de dood ons scheidt, dat duurt niet lang, he.",5
8,neutrality,Ze doet dit voor Bing. Ze zegt 't niet uit zic...,4
9,anger,Maar hoe... Dit kan niet. Kennelijk wel.,0


In [0]:
np.save('BERT_vectors', pred_vectors)

In [0]:
test_set

,subs,text,Label
0,neutrality,Ik heb iets anders ontdekt.,4
1,neutrality,"Jij, fiets alsjeblieft even naar het centrum v...",4
2,sadness,Ook voor ons. Wij kregen een kindje en... toen...,5
3,neutrality,Dank je wel.,4
4,neutrality,Het is ook belangrijk dat iemand 't hoofd koel...,4
5,anger,Als dat echt zo was dan had je me dit nooit ge...,0
6,neutrality,"Hee, ik heb een nieuwe locatie voor de bruilof...",4
7,sadness,"Tot de dood ons scheidt, dat duurt niet lang, he.",5
8,neutrality,Ze doet dit voor Bing. Ze zegt 't niet uit zic...,4
9,anger,Maar hoe... Dit kan niet. Kennelijk wel.,0


In [0]:
!wget https://github.com/wshuyi/info-5731-public/raw/master/imdb-sample.pickle

--2019-05-23 16:27:49--  https://github.com/wshuyi/info-5731-public/raw/master/imdb-sample.pickle
Resolving github.com (github.com)... 140.82.118.3
Connecting to github.com (github.com)|140.82.118.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/wshuyi/info-5731-public/master/imdb-sample.pickle [following]
--2019-05-23 16:27:50--  https://raw.githubusercontent.com/wshuyi/info-5731-public/master/imdb-sample.pickle
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3940753 (3.8M) [application/octet-stream]
Saving to: ‘imdb-sample.pickle’

imdb-sample.pickle  100%[===================>]   3.76M  --.-KB/s    in 0.04s   

2019-05-23 16:27:51 (107 MB/s) - ‘imdb-sample.pickle’ saved [3940753/3940753]



In [0]:
with open("imdb-sample.pickle", 'rb') as f:
  train, test = pickle.load(f)

In [0]:
dataset = pd.read_csv('refined_for_bert.csv', header=0, sep=';')[['subs', 'text']]

In [0]:
dataset.head()

,subs,text
0,anger,Kan ik hier nog betalen of hoe zit dat?
1,neutrality,Het is eigenlijk een heel algemeen en beschouw...
2,surprise,Hee. Is Sjoerd al weg?
3,happiness,Ik ben blij dat ze gelukkig is met Anton en ju...
4,neutrality,Ik weet het niet.


In [0]:
dataset = dataset.sample(len(dataset))

In [0]:
train, test = train_test_split(dataset, test_size=0.2)

In [0]:
myparam = {
    "DATA_COLUMN": "text",
    "LABEL_COLUMN": "subs",
    "LEARNING_RATE": 2e-5,
    "NUM_TRAIN_EPOCHS": 10,
    "bert_model_hub": 
}

In [0]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.INFO)

In [0]:
result